In [1]:
%matplotlib widget

import numpy as np
import ipympl
import matplotlib.pyplot as plt

In [2]:
fig, ax = plt.subplots()

x = np.linspace(0, 30, 30)

ax.plot(x, np.exp(0.2*x))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …